<a href="https://colab.research.google.com/github/PanosPanagopoulos/Image-Segmentation-Thesis/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
sys.path.insert(0,r'C:\Users\Panos\Desktop\thesis\SPIE2019_COURSE-master\SPIE2019_COURSE-master')

In [3]:
pip install SimpleITK

     |████████████████████████████████| 48.4 MB 1.4 MB/s 


In [4]:
import SimpleITK as sitk
import numpy as np
import os

%matplotlib nbagg
import gui

ModuleNotFoundError: ignored

In [8]:
import os

In [9]:
os.getcwd()

'/content'

In [5]:
# Let's import matplotlib so we can view images
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [6]:
folder = r'C:\Users\Panos\Desktop\thesis\data_pet'

In [7]:
reader = sitk.ImageFileReader()
reader.SetImageIO("NiftiImageIO")
reader.SetFileName(os.path.join(folder,'SUV\\Patient_01_AJ.nii'))
image = reader.Execute()

RuntimeError: ignored

In [ ]:
help(image)

In [ ]:
reader.SetFileName(os.path.join(folder,'Segmentations\\Patient_01_AJ_segm_fixed.nii'))
labels = reader.Execute()


In [ ]:
labels.GetSize()

In [ ]:
image.GetSize()

In [ ]:
image.GetDimension()

In [ ]:
print('Origin', image.GetOrigin())
print('Spacing', image.GetSpacing())
print('Direction', image.GetDirection())

In [ ]:
image.TransformIndexToPhysicalPoint((0,0,0))

In [ ]:
# view with Fiji
sitk.Show(image, title="grid using Show function", debugOn=True)

In [ ]:
# Change viewer, and display again.
image_viewer.SetApplication(
    '/Applications/ITK-SNAP.app/Contents/MacOS/ITK-SNAP')
image_viewer.Execute(grid_image)

In [ ]:
x = sitk.GetArrayFromImage(image)

In [ ]:
y = sitk.GetArrayFromImage(labels)

In [ ]:
y = np.where(y>0,1,0)

In [ ]:
len(np.unique(data))

In [ ]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

In [ ]:
data.shape

In [ ]:
%gui qt

In [ ]:
image_slice = image[:,:,6]
print('Slice size', image_slice.GetSize())

plt.imshow(sitk.GetArrayFromImage(image_slice), cmap='gray')

In [ ]:
dimension = image.GetDimension()

# Physical image size corresponds to the largest physical size in the training set, or any other arbitrary size.
reference_physical_size = np.zeros(dimension)
reference_physical_size[:] = [(sz-1)*spc if sz*spc>mx  else mx for sz,spc,mx in zip(image.GetSize(), 
                                                                                    image.GetSpacing(), 
                                                                                    reference_physical_size)]


In [ ]:
# Create the reference image with a zero origin, identity direction cosine matrix and dimension     
reference_origin = np.zeros(dimension)
reference_direction = np.identity(dimension).flatten()

In [ ]:
# Select arbitrary number of pixels per dimension, smallest size that yields desired results 
# or the required size of a pretrained network (e.g. VGG-16 224x224), transfer learning. This will 
# often result in non-isotropic pixel spacing.
reference_size = [128]*dimension 
reference_spacing = [ phys_sz/(sz-1) for sz,phys_sz in zip(reference_size, reference_physical_size) ]

In [ ]:
reference_image = sitk.Image(reference_size, image.GetPixelIDValue())
reference_image.SetOrigin(reference_origin)
reference_image.SetSpacing(reference_spacing)
reference_image.SetDirection(reference_direction)

In [ ]:
# Always use the TransformContinuousIndexToPhysicalPoint to compute an indexed point's physical coordinates as 
# this takes into account size, spacing and direction cosines. For the vast majority of images the direction 
# cosines are the identity matrix, but when this isn't the case simply multiplying the central index by the 
# spacing will not yield the correct coordinates resulting in a long debugging session. 
reference_center = np.array(reference_image.TransformContinuousIndexToPhysicalPoint(np.array(reference_image.GetSize())/2.0))

In [ ]:
def augment_images_spatial(original_image, reference_image, T0, T_aug, transformation_parameters,
                    output_prefix, output_suffix,
                    interpolator = sitk.sitkLinear, default_intensity_value = 0.0):
    '''
    Generate the resampled images based on the given transformations.
    Args:
        original_image (SimpleITK image): The image which we will resample and transform.
        reference_image (SimpleITK image): The image onto which we will resample.
        T0 (SimpleITK transform): Transformation which maps points from the reference image coordinate system 
            to the original_image coordinate system.
        T_aug (SimpleITK transform): Map points from the reference_image coordinate system back onto itself using the
               given transformation_parameters. The reason we use this transformation as a parameter
               is to allow the user to set its center of rotation to something other than zero.
        transformation_parameters (List of lists): parameter values which we use T_aug.SetParameters().
        output_prefix (string): output file name prefix (file name: output_prefix_p1_p2_..pn_.output_suffix).
        output_suffix (string): output file name suffix (file name: output_prefix_p1_p2_..pn_.output_suffix).
        interpolator: One of the SimpleITK interpolators.
        default_intensity_value: The value to return if a point is mapped outside the original_image domain.
    '''
    all_images = [] # Used only for display purposes in this notebook.
    for current_parameters in transformation_parameters:
        T_aug.SetParameters(current_parameters)        
        # Augmentation is done in the reference image space, so we first map the points from the reference image space
        # back onto itself T_aug (e.g. rotate the reference image) and then we map to the original image space T0.
        T_all = sitk.Transform(T0)
        T_all.AddTransform(T_aug)
        aug_image = sitk.Resample(original_image, reference_image, T_all,
                                  interpolator, default_intensity_value)
        sitk.WriteImage(aug_image, output_prefix + '_' + 
                        '_'.join(str(param) for param in current_parameters) +'_.' + output_suffix)
         
        all_images.append(aug_image) # Used only for display purposes in this notebook.
    return all_images # Used only for display purposes in this notebook.

In [ ]:
all_images = []
# Transform which maps from the reference_image to the current img with the translation mapping the image
# origins to each other.
transform = sitk.AffineTransform(dimension)
transform.SetMatrix(image.GetDirection())
transform.SetTranslation(np.array(image.GetOrigin()) - reference_origin)
# Modify the transformation to align the centers of the original and reference image instead of their origins.
centering_transform = sitk.TranslationTransform(dimension)
img_center = np.array(image.TransformContinuousIndexToPhysicalPoint(np.array(image.GetSize())/2.0))
centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - reference_center))
centered_transform = sitk.Transform(transform)
centered_transform = sitk.CompositeTransform([centering_transform, centered_transform])

In [ ]:
# Set the augmenting transform's center so that rotation is around the image center.
aug_transform = sitk.Similarity3DTransform()
aug_transform.SetCenter(reference_center)

In [ ]:
from utilities import parameter_space_regular_grid_sampling, similarity3D_parameter_space_regular_sampling, eul2quat

In [ ]:
transformation_parameters_list = similarity3D_parameter_space_regular_sampling(np.linspace(-np.pi/18.0,np.pi/18.0,3),
                                                                                   np.linspace(-np.pi/18.0,np.pi/18.0,3),
                                                                                   np.linspace(-np.pi/18.0,np.pi/18.0,3),
                                                                                   np.linspace(-10,10,3),
                                                                                   np.linspace(-10,10,3),
                                                                                   np.linspace(-10,10,3),
                                                                                   np.linspace(0.9,1.1,3))

In [ ]:
transformation_parameters_list[0]

In [ ]:
current_parameters = [0,0,np.pi/2,0,0,0,1]

In [ ]:
aug_transform.SetParameters(current_parameters)        
# Augmentation is done in the reference image space, so we first map the points from the reference image space
# back onto itself T_aug (e.g. rotate the reference image) and then we map to the original image space T0.
T_all = []
T_all.append(sitk.Transform(centered_transform))
T_all.append(aug_transform)
T_all = sitk.CompositeTransform(T_all)

In [ ]:
interpolator = sitk.sitkLinear
default_intensity_value = 0.0 

In [ ]:
aug_image = sitk.Resample(image, reference_image, T_all,
                          interpolator, default_intensity_value)

In [ ]:
sitk.WriteImage(aug_image, folder + '\\test.nii')

In [ ]:
def gen_chunk(in_img, in_mask, slice_count = 10, batch_size = 16):
    while True:
        img_batch = []
        mask_batch = []
        for _ in range(batch_size):
            s_idx = np.random.choice(range(in_img.shape[0]-slice_count))
            img_batch += [in_img[s_idx:(s_idx+slice_count)]]
            mask_batch += [in_mask[s_idx:(s_idx+slice_count)]]
        yield np.stack(img_batch, 0), np.stack(mask_batch, 0)       

In [ ]:
# training we use larger batch sizes with fewer slices
train_gen = gen_chunk(x, y)
x_out, y_out = next(train_gen)

# Model

In [ ]:
from keras.layers import ConvLSTM2D, Bidirectional, BatchNormalization, Conv3D, Cropping3D, ZeroPadding3D, Activation, Input
from keras.layers import MaxPooling3D, UpSampling3D, Conv3DTranspose, concatenate
from keras.models import Model

In [ ]:
in_layer = Input((None, None, None, 1))
bn = BatchNormalization()(in_layer)
cn1 = Conv3D(8, 
             kernel_size = (1, 5, 5), 
             padding = 'same',
             activation = 'relu')(bn)
cn2 = Conv3D(8, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'linear')(cn1)
bn2 = Activation('relu')(BatchNormalization()(cn2))

dn1 = MaxPooling3D((2, 2, 2))(bn2)
cn3 = Conv3D(16, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'linear')(dn1)
bn3 = Activation('relu')(BatchNormalization()(cn3))

dn2 = MaxPooling3D((1, 2, 2))(bn3)
cn4 = Conv3D(32, 
             kernel_size = (3, 3, 3),
             padding = 'same',
             activation = 'linear')(dn2)
bn4 = Activation('relu')(BatchNormalization()(cn4))

up1 = Conv3DTranspose(16, 
                      kernel_size = (3, 3, 3),
                      strides = (1, 2, 2),
                     padding = 'same')(bn4)

cat1 = concatenate([up1, bn3])

up2 = Conv3DTranspose(8, 
                      kernel_size = (3, 3, 3),
                      strides = (2, 2, 2),
                     padding = 'same')(cat1)

pre_out = concatenate([up2, bn2])

pre_out = Conv3D(1, 
             kernel_size = (1, 1, 1), 
             padding = 'same',
             activation = 'sigmoid')(pre_out)

pre_out = Cropping3D((1, 2, 2))(pre_out) # avoid skewing boundaries
out = ZeroPadding3D((1, 2, 2))(pre_out)
sim_model = Model(inputs = [in_layer], outputs = [out])
sim_model.summary()

In [ ]:
sim_model.predict(x_out).shape

In [ ]:
sim_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['binary_accuracy', 'mse'])
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('convlstm_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
sim_model.fit(train_gen, 
              batx
              epochs=1,
              steps_per_epoch = 100, 
              callbacks = callbacks_list)